In [163]:
import ipyvuetify as v
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from IPython.display import HTML
from IPython.core.display import Markdown
from IPython.display import display, clear_output
import json

### SQLytes Inventory Tracking

## Choose which requirement you want to view:

In [164]:
backend = 'http://127.0.0.1:5000/sqlytes/'
# output_area = widgets.Output()

In [165]:
class WarehouseWidgets:
    def __init__(self):
        self.output_area = widgets.Output()
        self.available_warehouses = self.getAvailableWarehouses()
        self.warehouses = self.getAllWarehouses().get('Warehouses')
        self.current_warehouse_index = 0
        self.all_warehouses_df = pd.DataFrame()
       
        self.warehouses_options_dropdown = widgets.Select(
            options=['All Warehouses info', 'All parts for a warehouse'],
            value='All Warehouses info',
            description='Search for:',
            rows=2,
        )
        
        #self.warehouses_options_dropdown.observe(self.warehouse_options_dropdown_eventhandler, names='value')

        self.warehouse_selection = widgets.Dropdown(
            options=self.available_warehouses,
            value=self.available_warehouses[0],
            description="Select wid:"
        )
        
        self.warehouse_selection.observe(self.on_selection_change)
        
        self.next_warehouse_button = widgets.Button(
            description="Next Warehouse",
            icon='check'
        )
        
    def getAvailableWarehouses(self):
        """Return all IDs of the warehouses in the Data Base."""
        response = requests.get(f'{backend}{"warehouse"}')
        # List to store all available wid's.
        wids = []
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            for record in data['Warehouses']:
                wids.append(record['wid'])
        else:
            print('Error {}: Could not get data from the backend.'.format(response.status_code))
        return wids
    
    def getAllWarehouses(self):
        """Return all warehouses in the Data Base."""
        response = requests.get(f'{backend}{"warehouse"}')
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            return data

        print(f'Error {response.status_code}: Could not get data from the backend.')
        return {}
    
    def displayNextWarehouse(self,_):
        self.current_warehouse_index = (self.current_warehouse_index+1) % len(self.warehouses)
        print(f"Displaying index {self.current_warehouse_index}")
        selected_warehouse = self.warehouses[self.current_warehouse_index]
        print(f"Displaying Warehouse ID: {selected_warehouse}")
        selected_warehouse_df = pd.DataFrame.from_dict(selected_warehouse, orient="index")
        self.all_warehouses_df = selected_warehouse_df
        with self.output_area:
            clear_output(wait=True)
            display(self.all_warehouses_df)
            
    def getWarehousePartsAPI(self,selected_warehouse):
        """Obtain all Parts for a warehouse.

        Args:
            wid (int): ID of the warehouse of which the parts are going to be showed.
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'warehouse/{selected_warehouse}/parts'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                all_parts_df = pd.DataFrame(data["WarehouseParts"])
                display(all_parts_df)
            elif response.status_code == 404:
                print('Error {}: Warehouse has no parts'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))
                
    def on_selection_change(self,change):
        if isinstance(change, int):
            # Directly received the selected value, not an event dictionary
            selected_warehouse = change
        elif change['type'] == 'change' and change['name'] == 'value':
            # Received an event dictionary, extract the selected value
            selected_warehouse = change['new']
        else:
            # Invalid change, do nothing
            return
        self.getWarehousePartsAPI(selected_warehouse)

In [166]:
# warehouseWidgets = WarehouseWidgets()

In [167]:
# warehouseWidgets.warehouses_options_dropdown.observe(warehouseWidgets.warehouse_options_handler,names='value')

In [168]:
# Display the dropdown initially
#display(warehouseWidgets.warehouses_options_dropdown)

In [169]:
def displayWarehouseWidgets():
    warehouseWidgets = WarehouseWidgets()
    warehouseWidgets.getWarehousePartsAPI(warehouseWidgets.warehouse_selection.value)
    warehouseWidgets.warehouse_selection.observe(warehouseWidgets.on_selection_change)
    display(warehouseWidgets.warehouse_selection)
    display(warehouseWidgets.output_area)

In [170]:
# display(warehouseWidgets.warehouses_options_dropdown)

In [171]:
# warehouseWidgets.displayNextWarehouse(_)

In [172]:
# # Attach the function to the button's click event
# button = warehouseWidgets.next_warehouse_button
# button.on_click(warehouseWidgets.displayNextWarehouse)

# # Display the widgets
# display(warehouseWidgets.output_area)
# display(button)

In [173]:
class SuppliedPartsWidgets:
    def __init__(self):
        self.output_area = widgets.Output()
        self.available_suppliers = self.getAvailableSuppliers()
        self.suppliers = self.getAllSuppliers()
        self.current_supplier_index = 0
        self.all_suppliers_df = pd.DataFrame()
       
        # self.suppliers_options_dropdown = widgets.Select(
        #     options=['All Suppliers info', 'All parts supplied by this supplier'],
        #     value='All Suppliers info',
        #     description='Search for:',
        #     rows=2,
        # )

        self.supplier_selection = widgets.Dropdown(
            options=self.available_suppliers,
            value=self.available_suppliers[0],
            description="Select sid:"
        )
        
        self.supplier_selection.observe(self.on_selection_change)
        
        # self.next_supplier_button = widgets.Button(
        #     description="Next Supplier",
        #     icon='check'
        # )
        
    def getAvailableSuppliers(self):
        """Return all IDs of the suppliers in the Data Base."""
        response = requests.get(f'{backend}{"supplier"}')
        # List to store all available sid's.
        sids = []
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            for record in data:
                sids.append(record['id'])
        else:
            print('Error {}: Could not get data from the backend.'.format(response.status_code))
        return sids
    
    def getAllSuppliers(self):
        """Return all suppliers in the Data Base."""
        response = requests.get(f'{backend}{"supplier"}')
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            return data

        print(f'Error {response.status_code}: Could not get data from the backend.')
        return {}
    
    # def displayNextSupplier(self,_):
    #     self.current_supplier_index = (self.current_supplier_index+1) % len(self.suppliers)
    #     print(f"Displaying index {self.current_supplier_index}")
    #     selected_supplier = self.suppliers[self.current_supplier_index]
    #     print(f"Displaying Warehouse ID: {selected_supplier}")
    #     selected_supplier_df = pd.DataFrame.from_dict(selected_supplier, orient="index")
    #     self.all_suppliers_df = selected_supplier_df
    #     with self.output_area:
    #         clear_output(wait=True)
    #         display(self.all_suppliers_df)
            
    def getSuppliedPartsAPI(self,selected_supplier):
        """Obtain all Parts for a warehouse.

        Args:
            wid (int): ID of the warehouse of which the parts are going to be showed.
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'supplies/{selected_supplier}'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                supplied_parts_df = pd.DataFrame(data)
                display(supplied_parts_df)
            elif response.status_code == 404:
                print('Error {}: Supplier does not supply any parts'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))
                
    def on_selection_change(self,change):
        if isinstance(change, int):
            # Directly received the selected value, not an event dictionary
            selected_supplier = change
        elif change['type'] == 'change' and change['name'] == 'value':
            # Received an event dictionary, extract the selected value
            selected_supplier = change['new']
        else:
            # Invalid change, do nothing
            return
        self.getSuppliedPartsAPI(selected_supplier)

In [174]:
def displaySuppliedParts():
    suppliedPartsWidgets = SuppliedPartsWidgets()
    suppliedPartsWidgets.getSuppliedPartsAPI(suppliedPartsWidgets.supplier_selection.value)
    suppliedPartsWidgets.supplier_selection.observe(suppliedPartsWidgets.on_selection_change)
    display(suppliedPartsWidgets.supplier_selection)
    display(suppliedPartsWidgets.output_area)

In [175]:
class TransactionWidgets:
    def __init__(self):
        self.output_area = widgets.Output()
        self.available_warehouses = self.getAvailableWarehouses()
        self.warehouses = self.getAllWarehouses()
        self.current_warehouse_index = 0
        self.all_transactions_df = pd.DataFrame()
       
        self.warehouse_options_dropdown = widgets.Select(
            options=['All Warehouses info', 'All transactions in this warehouse'],
            value='All Warehouses info',
            description='Search for:',
            rows=2,
        )

        self.warehouse_selection = widgets.Dropdown(
            options=self.available_warehouses,
            value=self.available_warehouses[0],
            description="Select wid:"
        )
        
        self.warehouse_selection.observe(self.on_selection_change)
        
        self.next_warehouse_button = widgets.Button(
            description="Next Warehouse",
            icon='check'
        )
        

    def getAvailableWarehouses(self):
        """
        Return all IDs of the warehouses in the DB
        """
        response = requests.get(f'{backend}{"warehouse"}')
        # List to store all available sid's.
        wids = []
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            for record in data['Warehouses']:
                wids.append(record['wid'])
        else:
            print('Error {}: Could not get data from the backend.'.format(response.status_code))
        return sorted(wids)

    
    def getAllWarehouses(self):
        """
        Return all suppliers in the Data Base.
        """
        response = requests.get(f'{backend}{"warehouse"}')
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            return data
        print(f'Error {response.status_code}: Could not get data from the backend.')
        return {}
                
    def getTransactionsPerWarehouseAPI(self, selected_warehouse):
        endpoint = f'transaction'
        response = requests.put(f'{backend}{endpoint}', data=json.dumps({"wid": selected_warehouse}), headers={"Content-Type": "application/json"})
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                supplied_parts_df = pd.DataFrame(data["Result"])
                display(supplied_parts_df)
            elif response.status_code == 404:
                print('Error {}: Warehouse does not have any transactions'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))
                
    def on_selection_change(self,change):
        if isinstance(change, int):
            # Directly received the selected value, not an event dictionary
            selected_warehouse = change
        elif change['type'] == 'change' and change['name'] == 'value':
            # Received an event dictionary, extract the selected value
            selected_warehouse    = change['new']
        else:
            # Invalid change, do nothing
            return
        self.getTransactionsPerWarehouseAPI(selected_warehouse)


In [176]:
def displayTransactionWidget():
    transactionWidget = TransactionWidgets()
    transactionWidget.getTransactionsPerWarehouseAPI(transactionWidget.warehouse_selection.value)
    transactionWidget.warehouse_selection.observe(transactionWidget.on_selection_change)
    display(transactionWidget.warehouse_selection)
    display(transactionWidget.output_area)

In [177]:
class PartPricesWidget:
    def __init__(self):
        self.table_output_area = widgets.Output()
        self.graph_output_area = widgets.Output()
        self.df = pd.DataFrame([])
                
            
    def getAllParts(self):
        endpoint = f'part'
        response = requests.get(f'{backend}{endpoint}')
        with self.table_output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                self.df = pd.DataFrame(data)
                display(self.df)
            elif response.status_code == 404:
                print('Error {}: Supplier does not supply any parts'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))

    def graphAllParts(self):
        names = []
        prices = []
        for _, row in self.df.iterrows():
            names.append(row['Name'])
            prices.append(row['msrp'])
        with self.graph_output_area:
            plt.figure(figsize=(16, 6))
            plt.bar(names, prices, color='red')
            plt.xlabel('Part Names')
            plt.ylabel('Part Prices ($)')
            plt.title('Price of all Parts')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()

In [178]:
def displayPartPriceWidget():
    partPricesWidget = PartPricesWidget()
    partPricesWidget.getAllParts()
    partPricesWidget.graphAllParts()
    display(partPricesWidget.table_output_area)
    display(partPricesWidget.graph_output_area)

In [179]:
class MostRacksWidgets:
    def __init__(self):
        self.output_area = widgets.Output()

        self.current_supplier_index = 0
        self.all_suppliers_df = pd.DataFrame()
       

    def get_most_racks(self):
        """Obtain Top 10 Warehouses with the most Racks.

        Args:
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'most/rack'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                all_parts_df = pd.DataFrame(data["Top Racks per Warehouse"])
                display(all_parts_df)
                
                                #Data for Bar Graph
                incoming = [entry["Rack Count"]  for entry in data["Top Racks per Warehouse"]]
                warehouse = [entry["Warehouse"] for entry in data["Top Racks per Warehouse"]]
                
                #Creating Bar Graph
                plt.bar(warehouse, incoming, color='#95d0ed')
                plt.xlabel('\n Warehouse')
                plt.ylabel('Rack Count')
                plt.title('Racks per Warehouse')
                plt.xticks(rotation=45, ha='right')  #Rotate
                plt.tight_layout()  # Adjust to prevent clipping
                plt.show()
                
                
            elif response.status_code == 404:
                print('Error {}: Warehouses have no racks, check your data.'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))

In [ ]:

class MostIncomingTransactions:
    def __init__(self):
        self.output_area = widgets.Output()

        self.current_supplier_index = 0
        self.all_suppliers_df = pd.DataFrame()
       

    def get_most_incoming(self):
        """Obtain Top 5 Warehouses with the most incoming transactions.

        Args:
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'most/incoming'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                all_parts_df = pd.DataFrame(data["Most Incoming Transactions"])
                display(all_parts_df)
                
                #Data for Bar Graph
                incoming = [entry["Total Incoming Transactions"]  for entry in data["Most Incoming Transactions"]]
                warehouse = [entry["Warehouse"] for entry in data["Most Incoming Transactions"]]
                
                #Creating Bar Graph
                plt.bar(warehouse, incoming, color='#95d0ed')
                plt.xlabel('\n Warehouse')
                plt.ylabel('Incoming')
                plt.title('Incoming Transactions per Warehouse')
                plt.xticks(rotation=45, ha='right')  #Rotate
                plt.tight_layout()  # Adjust to prevent clipping
                plt.show()
                
            elif response.status_code == 404:
                print('Error {}: Warehouses have no racks, check your data.'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))       

In [ ]:

class MostExchangesWidget:
    def __init__(self):
        self.output_area = widgets.Output()

        self.current_supplier_index = 0
        self.all_suppliers_df = pd.DataFrame()
       

    def get_most_delivered(self):
        """Obtain Top 5 Warehouses that deliver the most exchanges.

        Args:
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'most/deliver'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                all_parts_df = pd.DataFrame(data["Most Transfers"])
                display(all_parts_df)
                
                #Data for Bar Graph
                transfers = [entry["Total Transfers"]  for entry in data["Most Transfers"]]
                warehouse = [entry["Warehouse"] for entry in data["Most Transfers"]]
                
                #Creating Bar Graph
                plt.bar(warehouse, transfers, color='#95d0ed')
                plt.xlabel('\n Warehouse')
                plt.ylabel('Exchanges')
                plt.title('Exchanges per Warehouse')
                plt.xticks(rotation=45, ha='right')  #Rotate
                plt.tight_layout()  # Adjust to prevent clipping
                plt.show()   
                
            elif response.status_code == 404:
                print('Error {}: Warehouses have no racks, check your data.'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))

In [ ]:

class MostUserTransactionsWidget:
    def __init__(self):
        self.output_area = widgets.Output()

        self.current_supplier_index = 0
        self.all_suppliers_df = pd.DataFrame()
       

    def get_most_users(self):
        """Obtain Top 3 Users that made the most transactions..

        Args:
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'most/transactions'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                all_parts_df = pd.DataFrame(data["Top User Transactions"])
                display(all_parts_df)
                
                #Data for Bar Graph
                transactions = [entry["Transaction Count"]  for entry in data["Top User Transactions"]]
                name = [entry["First Name"] + " " + entry["Last Name"] for entry in data["Top User Transactions"]]
                
                #Creating Bar Graph
                plt.bar(name, transactions, color='#95d0ed')
                plt.xlabel('\n Users')
                plt.ylabel('Total Transactions')
                plt.title('Transactions per User')
                plt.xticks(rotation=45, ha='right')  #Rotate
                plt.tight_layout()  # Adjust to prevent clipping
                plt.show()   
                
            elif response.status_code == 404:
                print('Error {}: Warehouses have no racks, check your data.'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))   

In [ ]:

class LeastOutTransactionsWidget:
    def __init__(self):
        self.output_area = widgets.Output()

        self.current_supplier_index = 0
        self.all_suppliers_df = pd.DataFrame()
       

    def get_least_outgoing(self):
        """Obtain Top 3 warehouses with the least outgoing transactions.

        Args:
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'least/outgoing'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                all_parts_df = pd.DataFrame(data["Least Outgoing Transactions"])
                display(all_parts_df)
                
                #Data for Bar Graph
                outgoing = [entry["Total Outgoing Transactions"]  for entry in data["Least Outgoing Transactions"]]
                warehouse = [entry["Warehouse"] for entry in data["Least Outgoing Transactions"]]
                
                #Creating Bar Graph
                plt.bar(warehouse, outgoing, color='#95d0ed')
                plt.xlabel('\n Warehouse')
                plt.ylabel('Outgoing Transactions')
                plt.title('Least Outgoing Transaction per Warehouse')
                plt.xticks(rotation=45, ha='right')  #Rotate
                plt.tight_layout()  # Adjust to prevent clipping
                plt.show()   
                
            elif response.status_code == 404:
                print('Error {}: Warehouses have no racks, check your data.'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))

In [ ]:
class MostCityTransactionsWidget:
    def __init__(self):
        self.output_area = widgets.Output()

        self.current_supplier_index = 0
        self.all_suppliers_df = pd.DataFrame()
       

    def get_most_cities(self):
        """Obtain Top 3 warehouses' cities with the most transactions.

        Args:
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'most/city'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                all_parts_df = pd.DataFrame(data["Most Transactions per City"])
                display(all_parts_df)
                
                #Data for Bar Graph
                transaction_Counts = [entry["Total Transactions"]  for entry in data["Most Transactions per City"]]
                cities = [entry["Warehouse City"] for entry in data["Most Transactions per City"]]
                
                #Creating Bar Graph
                plt.bar(cities, transaction_Counts, color='#95d0ed')
                plt.xlabel('\n Cities')
                plt.ylabel('Total Transactions')
                plt.title('Total Transactions per City')
                plt.xticks(rotation=45, ha='right')  #Rotate
                plt.tight_layout()  # Adjust to prevent clipping
                plt.show()   
                
                
            elif response.status_code == 404:
                print('Error {}: Warehouses have no racks, check your data.'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))

In [3]:
def displayGlobalStatisticsWidget():
    
    display(Markdown("## Top 10 Warehouses with the most Racks"))
    mostRacksWidgets = MostRacksWidgets()
    mostRacksWidgets.get_most_racks()
    display(mostRacksWidgets.output_area)
    
    display(Markdown("## Top 5 Warehouses with the most Incoming Transactions"))
    mostIncomingTransactions = MostIncomingTransactions()
    mostIncomingTransactions.get_most_incoming()
    display(mostIncomingTransactions.output_area)

    display(Markdown("## Top 5 Warehouses that deliver the most Exchanges"))
    mostExchangesWidget = MostExchangesWidget()
    mostExchangesWidget.get_most_delivered()
    display(mostExchangesWidget.output_area)
    
    display(Markdown("## Top 3 Users that made the most Transactions"))
    mostUserTransactionsWidget = MostUserTransactionsWidget()
    mostUserTransactionsWidget.get_most_users()
    display(mostUserTransactionsWidget.output_area)
    
    display(Markdown("## Top 3 Warehouses with the least Outgoing Transactions"))
    leastOutTransactionsWidget = LeastOutTransactionsWidget()
    leastOutTransactionsWidget.get_least_outgoing()
    display(leastOutTransactionsWidget.output_area)
    
    display(Markdown("## Top 3 Warehouses' Cities with the most Transactions"))
    mostCityTransactionsWidget = MostCityTransactionsWidget()
    mostCityTransactionsWidget.get_most_cities()
    display(mostCityTransactionsWidget.output_area)

In [ ]:
class MenuWidgets:
    def __init__(self):
        self.output_area = widgets.Output()
        self.specs = ["Local Statistics", "Global Statistics", "Transactions in Warehouse", "Part Prices", "Parts Supplied by Suppliers", "Parts in Warehouse"]
        
        self.spec_selection = widgets.Dropdown(
            options=self.specs,
            value = self.specs[0],
            description="Select view:"
        )
        
        self.spec_selection.observe(self.on_selection_change)
        
    def getSpec(self, selected_spec):
        with self.output_area:            
            clear_output(wait=True)  # Clear the previous output
            
            display_functions = {
                
                self.specs[1]: displayGlobalStatisticsWidget,
                self.specs[2]: displayTransactionWidget,
                self.specs[3]: displayPartPriceWidget,
                self.specs[4]: displaySuppliedParts,
                self.specs[5]: displayWarehouseWidgets,
            }
        
            display_function = display_functions.get(selected_spec, None)
        
            if display_function is not None:
                display_function()
            else:
                print(selected_spec)

                
            
                
    def on_selection_change(self,change):
        if isinstance(change, int):
            # Directly received the selected value, not an event dictionary
            selected_supplier = change
        elif change['type'] == 'change' and change['name'] == 'value':
            # Received an event dictionary, extract the selected value
            selected_supplier = change['new']
        else:
            # Invalid change, do nothing
            return
        self.getSpec(selected_supplier)

In [180]:
menuWidget = MenuWidgets()
menuWidget.getSpec(menuWidget.spec_selection.value)
menuWidget.spec_selection.observe(menuWidget.on_selection_change)
display(menuWidget.spec_selection)
display(menuWidget.output_area)

Dropdown(description='Select view:', options=('Local Statistics', 'Global Statistics', 'Transactions in Wareho…

AttributeError: 'MenuWidgets' object has no attribute 'output_area'